In [ ]:


import numpy as np 
import pandas as pd 

import os
print(os.listdir("../input"))


In [ ]:
df=pd.read_csv('../input/bank.csv')

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import datasets
from io import StringIO
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
%matplotlib inline

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

#categorical variables

job, marital ,education, default, housing ,loan ,contact,month ,poutcome,deposit

#numerical variables

age, balance,day,duration,campaign,pdays, previous





In [ ]:

y=df['deposit']
X=df
y=y.to_frame()
y

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=101)
print(y.shape)
print(X.shape)
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

In [ ]:
X_train.columns

In [ ]:
sns.pairplot(X_train,diag_kind='hist')

In [ ]:
y.value_counts()

In [ ]:
combine_=[y_train,y_test]
depositmapping={'yes':1,'no': 0}
for dt in combine_:
   dt['deposit']=dt['deposit'].map(depositmapping)
y_train

In [ ]:
#assigning 1 to yes and 0 to no in deposit
combine=[X_train,X_test]
depositmapping={'yes':1,'no': 0}
for dt in combine:
   dt['deposit']=dt['deposit'].map(depositmapping)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:


X_train[['job','deposit']].groupby('job',as_index=False).mean().sort_values('deposit',ascending=False)

we see that students ,retired and umemployed people are more likely to deposit.So we assign 1 to students and 2 to retired ,3 to unemployed and 0 to others

In [ ]:
for df in combine:
    df['job']=df['job'].replace(['management','technician','unknown','admin.','housemaid','self-employed','services',
                                'blue-collar','entrepreneur'],'rare',regex=True)
jobmapping={'student':1,'retired':2,'unemployed':3,'rare':0}
for df in combine:
    df['job']=df['job'].map(jobmapping)
    

In [ ]:


X_train[['marital','deposit']].groupby('marital',as_index=False).mean().sort_values('deposit',ascending=False)

We see that the marital status is of not much significance when deciding whether the person will deposit or not.
Therefore we eliminate this parameter

In [ ]:

X_train[['education','deposit']].groupby('education',as_index=False).mean().sort_values('deposit',ascending=False)

# from the above data we can say that the more educated the person is the more likely he is to deposit.
Therefore we assign 1 to primary , 2 to secondary , 3 to tertiary and 0 to unknown





In [ ]:
educationmapping={'primary':1,'secondary':2,'tertiary':3,'unknown':0}
for df in combine:
    df['education']=df['education'].map(educationmapping)

In [ ]:
#analysing the variable 'default'

X_train[['default','deposit']].groupby('default',as_index=False).mean().sort_values('deposit',ascending=False)

We can see that who did not have credit in default ,deposited more.
We assign 1 to 'no' and 0 to 'yes'

In [ ]:
defaultmapping={'no':1,'yes':0}
for df in combine:
    df['default']=df['default'].map(defaultmapping)

In [ ]:
#analysing the variable 'loan'

X_train[['loan','deposit']].groupby('loan',as_index=False).mean().sort_values('deposit',ascending=False)

we can see that people who did not have personal loan ,deposited more.
We assign 1 to 'no' and 0 to 'yes'

In [ ]:
loanmapping={'no':1,'yes':0}
for df in combine:
    df['loan']=df['loan'].map(defaultmapping)

In [ ]:
#analysing the variable 'contact'

X_train[['contact','deposit']].groupby('contact',as_index=False).mean().sort_values('deposit',ascending=False)

We can see that the way the customer was contacted has little to do with whether the person has deposited,so we drop this variable.


In [ ]:
#analysing the variable 'month'

X_train[['month','deposit']].groupby('month',as_index=False).mean().sort_values('deposit',ascending=False)

We will classify months into three categories.
Let us assign 2 to mar.dec.sep.oct and 1 to 'apr,feb,aug,jun and 0 to nov,jul,jan,may

In [ ]:
for df in combine:
    df['month']=df['month'].replace(['mar','dec','sep','oct'],2,regex=True)
    df['month']=df['month'].replace(['apr','feb','aug','jun'],1,regex=True)
    df['month']=df['month'].replace(['nov','jul','jan','may'],0,regex=True)

In [ ]:
#analysing the variable 'poutcome'

X_train[['poutcome','deposit']].groupby('poutcome',as_index=False).mean().sort_values('deposit',ascending=False)

We can see that people who have deposited previously ,deposited again.
We assign 2 to sucess,1 to other and 0 to failure an d unknown.


In [ ]:
poutcomemapping={'success':2,'other':1,'failure':0,'unknown':0}

for df in combine:
    df['poutcome']=df['poutcome'].map(poutcomemapping)

**Analyzing numerical variables**

age, balance,day,duration,campaign,pdays, previous


In [ ]:
#Analysing the variable 'age'
sns.distplot(X_train['age'])

In [ ]:
#Let us calculate skewness and kurtosis.
from scipy.stats import kurtosis
from scipy.stats import skew

K=kurtosis(X_train['age'])
s=skew(X_train['age'])
print('k:',K)
print('s:',s)

From the values of k and s we an say that the data is moderately skewed.Let us apply log trans to reduce skewness.

In [ ]:
for df in combine:
    df['age_trns']=df['age'].apply(np.log)
    
sns.distplot(X_train['age_trns'])

In [ ]:
K=kurtosis(X_train['age_trns'])
s=skew(X_train['age_trns'])
print('k:',K)
print('s:',s)

In [ ]:
plt.figure()
x=df[df['deposit']==1]['age_trns']
y=df[df['deposit']==0]['age_trns']
plt.hist(x,bins=20,alpha=0.5,label='deposited')
plt.hist(y,bins=20,alpha=0.5,label='did not deposit')
plt.legend()
plt.show()

We can draw the following conclusions from the above graph.
1)Both the hist are almost overlap each other
2)We see that younger people and older people are more likely to make a deposit.
We can validate this by forming bins and analysing.

In [ ]:
for df in combine:
    df['age_bands']=pd.cut(df['age_trns'],bins=5,precision=1)
X_train[['age_bands','deposit']].groupby('age_bands',as_index=False).mean().sort_values('deposit',ascending=False)

#We assign people age_bands 1,2,3-1 and people in age_bands 0 ,4-2
for df in combine:
        df.loc[df['age_trns']>=4.2,'age_trns']=2
        df.loc[(df['age_trns']>=2.9) & (df['age_trns']<3.2),'age_trns']=2
        df.loc[(df['age_trns']>=3.2) & (df['age_trns']<4.2),'age_trns']=1
X_train.head()

In [ ]:
#analyzing the variable balance
plt.figure(figsize=(12,8))
sns.distplot(X_train['balance'])
K_bal=kurtosis(X_train['balance'])
s_bal=skew(X_train['balance'])
print('k:',K_bal)
print('s:',s_bal)


In [ ]:
#We see that this variable has very high k and s.So let us apply log trans and see.

for df in combine:
    df['bal_trns']=df['balance'].apply(np.cbrt)
    
sns.distplot(X_train['bal_trns'])

K_trns_bal=kurtosis(X_train['bal_trns'])
s_trns_bal=skew(X_train['bal_trns'])
print('k:',K_trns_bal)
print('s:',s_trns_bal)

In [ ]:
#We see that our s and k values decreased significantly.


In [ ]:
x=df[df['deposit']==1]['bal_trns']
y=df[df['deposit']==0]['bal_trns']
plt.figure(figsize=(12,8))
plt.hist(x,bins=10,alpha=0.5,label='deposited')
plt.hist(y,bins=10,alpha=0.5,label='did not deposit')
plt.legend()

The graph doesn't seem very useful to derive any conclusions.
let us divide balance into bands.

In [ ]:
X_train['bal_bands']=pd.cut(X_train['bal_trns'],bins=10)
X_train[['bal_bands','deposit']].groupby('bal_bands',as_index=False).mean().sort_values('bal_bands',ascending=False)

We see that the higher the bank balance of the person is ,he / she is more likely to make to a deposit

In [ ]:
#analysing the variable day
sns.distplot(X_train['day'])

In [ ]:
x=df[df['deposit']==1]['day']
y=df[df['deposit']==0]['day']
plt.figure(figsize=(12,8))
plt.hist(x,bins=31,alpha=0.5,label='deposited')
plt.hist(y,bins=31,alpha=0.5,label='did not deposit')
plt.legend()

We see that the data is almost symmetrically distributed and We see that the day on which the customer was contacted also doesn't have much influence on whether the person deposited or not although we might expect that the customer might deposit at the starting of every month as people usually tend have more money in hand in the beginning of every month.

In [ ]:
#Analysing the variable duration

sns.distplot(X_train['duration'])
K_dur=kurtosis(X_train['duration'])
s_dur=skew(X_train['duration'])
print('k:',K_dur)
print('s:',s_dur)

In [ ]:
#We see that the data is highly skewed.Let us apply cbrt tranformation.
for df in combine:
    df['dur_trns']=df['duration'].apply(np.cbrt)
    
sns.distplot(X_train['dur_trns'])

K_trns_dur=kurtosis(X_train['dur_trns'])
s_trns_dur=skew(X_train['dur_trns'])
print('k:',K_trns_dur)
print('s:',s_trns_dur)


In [ ]:
#We will not include the variable duartion as we nees to predict whether the customer will make a deopsit before making a call.

In [ ]:
#analysing the variable campaign
sns.distplot(X_train['campaign'])
K_cam=kurtosis(X_train['campaign'])
s_cam=skew(X_train['campaign'])
print('k:',K_cam)
print('s:',s_cam)

In [ ]:
for df in combine:
    df['cam_trns']=df['campaign'].apply(np.log)
    
sns.distplot(X_train['cam_trns'])

K_trns_cam=kurtosis(X_train['cam_trns'])
s_trns_cam=skew(X_train['cam_trns'])
print('k:',K_trns_cam)
print('s:',s_trns_cam)

In [ ]:
x=df[df['deposit']==1]['cam_trns']
y=df[df['deposit']==0]['cam_trns']
plt.figure(figsize=(12,8))
plt.hist(x,bins=5,alpha=0.5,label='deposited')
plt.hist(y,bins=5,alpha=0.5,label='did not deposit')
plt.legend()

In [ ]:
X_train['cam_bands']=pd.cut(X_train['cam_trns'],bins=10)
X_train[['cam_bands','deposit']].groupby('cam_bands',as_index=False).mean().sort_values('deposit',ascending=False)

In [ ]:
We see that the prob of depositing inc with inc in cam_trns

In [ ]:
#analysing the variable pdays
sns.distplot(X_train['pdays'])
K_pd=kurtosis(X_train['pdays'])
s_pd=skew(X_train['pdays'])
print('k:',K_pd)
print('s:',s_pd)

In [ ]:
for df in combine:
    df['pd_trns']=df['pdays'].apply(np.cbrt)
    
sns.distplot(X_train['pd_trns'])

K_trns_pd=kurtosis(X_train['pd_trns'])
s_trns_pd=skew(X_train['pd_trns'])
print('k:',K_trns_pd)
print('s:',s_trns_pd)

In [ ]:
x=df[df['deposit']==1]['pd_trns']
y=df[df['deposit']==0]['pd_trns']
plt.figure(figsize=(12,8))
plt.hist(x,bins=10,alpha=0.5,label='deposited')
plt.hist(y,bins=10,alpha=0.5,label='did not deposit')
plt.legend()

In [ ]:
X_train['p_bands']=pd.cut(X_train['pd_trns'],bins=10)
X_train[['p_bands','deposit']].groupby('p_bands',as_index=False).mean().sort_values('deposit',ascending=False)

In [ ]:
#As we cannot see any trend we drop this variable for now and later on see if we can use it in multivariable analysis.

In [ ]:
#analysing the variable previous
sns.distplot(X_train['previous'])
K_pre=kurtosis(X_train['previous'])
s_pre=skew(X_train['previous'])
print('k:',K_pre)
print('s:',s_pre)




In [ ]:
for df in combine:
    df['pre_trns']=df['previous'].apply(np.cbrt)
    
sns.distplot(X_train['pre_trns'])

K_trns_pre=kurtosis(X_train['pre_trns'])
s_trns_pre=skew(X_train['pre_trns'])
print('k:',K_trns_pre)
print('s:',s_trns_pre)

In [ ]:
X_train['pre_bands']=pd.cut(X_train['pre_trns'],bins=10)
X_train[['pre_bands','deposit']].groupby('pre_bands',as_index=False).mean().sort_values('deposit',ascending=False)

In [ ]:
#no significant trend is noticied so let us drop this parameter for now

In [ ]:
X_train.columns

In [ ]:
X_train.head()

In [ ]:
X_final_train=X_train[['job','loan','month','poutcome','age_trns','bal_trns','cam_trns','dur_trns']]
X_final_test=X_test[['job','loan','month','poutcome','age_trns','bal_trns','cam_trns','dur_trns']]

In [ ]:
X_train.columns

#Modelling using decision trees

In [ ]:
from sklearn.tree import  DecisionTreeClassifier

dt=DecisionTreeClassifier(random_state=101)
dt.fit(X_final_train,y_train)
predict=dt.predict(X_final_test)
accuracy_test=round(dt.score(X_final_test,y_test)*100,2)
accuracy_train=round(dt.score(X_final_train,y_train)*100,2)

print('train accuracy of decision tree classifier',accuracy_train)
print('test accuracy of decision tree classifier',accuracy_test)
  

In [ ]:
k_plot=[]
t_plot=[]

for k in range(1,10,1):
     dt=DecisionTreeClassifier(max_depth=k,random_state=101)
     dt.fit(X_final_train,y_train)
     predict=dt.predict(X_final_test)
     accuracy_test=round(dt.score(X_final_test,y_test)*100,2)
     accuracy_train=round(dt.score(X_final_train,y_train)*100,2)
     print(k)
     #print('train accuracy of decision tree classifier',accuracy_train)
     #print('test accuracy of decision tree classifier',accuracy_test)
     k_plot.append(accuracy_test)
     t_plot.append(accuracy_train)

In [ ]:
fig,axes=plt.subplots(1,1,figsize=(12,8))

axes.set_xticks(range(1,10,1))
k=range(1,10,1)
plt.plot(k,k_plot)
plt.plot(k,t_plot,'r')

We see that accuracy has increased till max_depth=5 and beyond that the increase in accuracy is not much.So let us se the depth to 5

In [ ]:
dt=DecisionTreeClassifier(max_depth=5,random_state=101)
dt.fit(X_final_train,y_train)
predict=dt.predict(X_final_test)
accuracy_test=round(dt.score(X_final_test,y_test)*100,2)
accuracy_train=round(dt.score(X_final_train,y_train)*100,2)

print('train accuracy of decision tree classifier',accuracy_train)
print('test accuracy of decision tree classifier',accuracy_test)

In [ ]:
#Using Random forest classifier
from sklearn.ensemble import RandomForestClassifier
k_plot=[]
t_plot=[]
for k in range(10,100,10):
    rfc = RandomForestClassifier(n_estimators=k)
    rfc.fit(X_final_train, y_train)
    predict=rfc.predict(X_final_test)
    accuracy_test=round(rfc.score(X_final_test,y_test)*100,2)
    accuracy_train=round(rfc.score(X_final_train,y_train)*100,2)
    k_plot.append(accuracy_test)
    t_plot.append(accuracy_train)

fig,axes=plt.subplots(1,1,figsize=(12,8))

axes.set_xticks(range(10,100,10))
k=range(10,100,10)
plt.plot(k,k_plot)
plt.plot(k,t_plot,'r')
    


#We see that increasing the number of trees beyond 30 ,doesn't increase our test accuracy significantly, so 
we use set the number of trees=30

In [ ]:
rfc = RandomForestClassifier(n_estimators=30)
rfc.fit(X_final_train, y_train)
predict=dt.predict(X_final_test)
accuracy_test=round(dt.score(X_final_test,y_test)*100,2)
accuracy_train=round(dt.score(X_final_train,y_train)*100,2)
print(accuracy_test)
print(accuracy_train)